<a href="https://colab.research.google.com/github/dAn-solution/competition/blob/main/Prob_kiva_021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Kiva／クラウドファンディングの資金調達額予測
- LightGBM を実施

### Google Driveのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'My Drive'

Mounted at /content/drive
/content/drive/My Drive


### ライブラリのインストール、インポート

In [ ]:
# カレントディレクトリを変更
import os
os.chdir('/content/drive/My Drive/Probdata/kiva/')
print(os.getcwd())

/content/drive/My Drive/Probdata/kiva


In [ ]:
class Config():
    root_path = './'
    input_path = os.path.join(root_path, 'input')
    output_path = os.path.join(root_path, 'output')
    result_path = os.path.join(root_path, 'result')
    bert_model_name = 'bert-base-uncased'
    seed = 42
    constant = 25
    debug = False

In [ ]:
import pandas as pd
import numpy as np
import datetime
import warnings

import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error

import lightgbm as lgb

### データの読み込み

In [ ]:
train_df = pd.read_csv(f'{Config.input_path}/train.csv')
test_df = pd.read_csv(f'{Config.input_path}/test.csv')
sample_df = pd.read_csv(f'{Config.input_path}/sample_submission.csv')

train_011_df = pd.read_csv(f'{Config.result_path}/train_011.csv')
test_011_df = pd.read_csv(f'{Config.result_path}/test_011.csv')
train_012_df = pd.read_csv(f'{Config.result_path}/train_012.csv')
test_012_df = pd.read_csv(f'{Config.result_path}/test_012.csv')
# train_013_df = pd.read_csv(f'{Config.result_path}/train_013.csv')
# test_013_df = pd.read_csv(f'{Config.result_path}/test_013.csv')
# train_014_df = pd.read_csv(f'{Config.result_path}/train_014.csv')
# test_014_df = pd.read_csv(f'{Config.result_path}/test_014.csv')
train_015_df = pd.read_csv(f'{Config.result_path}/train_015.csv')
test_015_df = pd.read_csv(f'{Config.result_path}/test_015.csv')
train_016_df = pd.read_csv(f'{Config.result_path}/train_016.csv')
test_016_df = pd.read_csv(f'{Config.result_path}/test_016.csv')
train_020_df = pd.read_csv(f'{Config.result_path}/train_020.csv')
test_020_df = pd.read_csv(f'{Config.result_path}/test_020.csv')

## LGBMの実行

In [ ]:
target_column = "LOAN_AMOUNT"
y = train_df[target_column]

In [ ]:
# 合体
# X = pd.concat([train_011_df, train_012_df, train_013_df, train_014_df, train_015_df, train_016_df, train_020_df], axis=1)
# test_X = pd.concat([test_011_df, test_012_df, test_013_df, test_014_df, test_015_df, test_016_df, test_020_df], axis=1)
X = pd.concat([train_011_df, train_012_df, train_015_df, train_016_df, train_020_df], axis=1)
test_X = pd.concat([test_011_df, test_012_df, test_015_df, test_016_df, test_020_df], axis=1)

In [ ]:
print(X)

                 0            0       0            0           0
0       305.805124   188.879689   241.0   391.633822   295.06458
1       629.991425   821.488494   563.0   731.540524   903.21484
2       658.631480   940.125007   580.0   801.048825   735.61460
3       473.877850  1079.946468   490.5   764.677435   578.70750
4       915.806823  1218.223435   621.0  2219.072167  1860.70300
...            ...          ...     ...          ...         ...
45665   158.493614   235.146212   150.5   206.406165   184.21630
45666  1469.051171  1281.554173  1072.0  1367.677873   699.49677
45667  5271.795639  1539.576415  4609.5  3264.882560  7946.42100
45668   730.270023  1288.558699   887.5   883.459517   570.45770
45669  1617.169716  1175.277983  2227.0  2553.074569  1719.25100

[45670 rows x 5 columns]


In [ ]:
# trainデータを訓練用とテスト用に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = Config.seed)

In [ ]:
# LGBM用のパラメータ設定
lgb_params = {
    "application" : "regression_l1",
    'n_estimators': 500, 'learning_rate': 0.08,
    "metric" : "l1" #mae
}

# lgb_params = {
#     "application" : "regression_l1",
#     # 'objective': 'regression',
#     'importance_type': 'gain',
#     'metric': 'l1', # mae
#     'seed': Config.seed,
#     'n_jobs': -1,
#     'verbose': 0,

#     'n_estimators': 10000, 'learning_rate': 0.09,
#     'boosting_type': 'gbdt',
#     'subsample': 0.5, 'subsample_freq': 1,
#     'colsample_bytree': 0.5,
#     'num_leaves': 256, 'max_depth': -1, 'min_child_samples': 255,
#     'max_bin': 256,
# }

In [ ]:
# 学習/予測

# groups = X_train['LOAN_ID']
y_oof = np.zeros(len(y_train))
meta_preds = []
y_preds = []
kf = StratifiedKFold(n_splits=5)
for fold, (tr_idx, vl_idx) in enumerate(kf.split(X_train, y_train)):
    x_tr_fold = X.iloc[tr_idx]
    y_tr_fold = y.iloc[tr_idx]
    x_vl_fold = X.iloc[vl_idx]
    y_vl_fold = y.iloc[vl_idx]

    model = lgb.LGBMRegressor(**lgb_params)
    model.fit(
        x_tr_fold, y_tr_fold,
        eval_set=(x_vl_fold, y_vl_fold),
        eval_metric='l1',
        verbose=False,
        early_stopping_rounds=100,
    )
    y_oof[vl_idx] = model.predict(x_vl_fold)
    meta_preds.append(model.predict(X))
    y_preds.append(model.predict(test_X))
    print(
        f'fold {fold} score:', mean_absolute_error(y_oof[vl_idx], y_vl_fold)
    )

model = lgb.LGBMRegressor(**lgb_params)
model.fit(X_train, y_train, eval_set=(X_test, y_test),
          eval_metric='l1', verbose=False, early_stopping_rounds=100,
          )
print('X score:', mean_absolute_error(model.predict(X), y))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


fold 0 score: 112.86312467846346
fold 1 score: 112.49127091127937
fold 2 score: 108.97780973829114
fold 3 score: 111.26473993454101
fold 4 score: 113.64733806412367
X score: 145.51076196079694


In [ ]:
# メタモデルへの引き渡し
train_021_df = pd.DataFrame(np.mean(meta_preds, axis=0))
train_021_df.to_csv(os.path.join(Config.result_path, "train_021.csv"), index=False)

test_021_df = pd.DataFrame(np.mean(y_preds, axis=0))
test_021_df.to_csv(os.path.join(Config.result_path, "test_021.csv"), index=False)

### 提出データの作成

In [ ]:
result = np.mean(y_preds, axis=0)
for i in range(len(result)):
    a, b = divmod(result[i], Config.constant)
    if b > (Config.constant/2):
        b = 1
    else:
        b = 0
    result[i] = (a + b) * Config.constant

In [ ]:
sample_df["LOAN_AMOUNT"] = result
print(sample_df)

       LOAN_ID  LOAN_AMOUNT
0      2041445        325.0
1      1944435        425.0
2      2083354        350.0
3      1993565        475.0
4      2064272        450.0
...        ...          ...
91817  1993862        300.0
91818  2015070        225.0
91819  1950349        125.0
91820  1921580       1325.0
91821  1976733        325.0

[91822 rows x 2 columns]


In [ ]:
# submitファイルのファイル名に利用する作成時刻の取得

now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=1)))
date_time = "{0:%Y%m%d_%H%M}".format(now)

In [ ]:
# submitファイルの出力(Google Driveに出力）
# 形式：submit_yyyymmdd_hhmm.csv

sample_df.to_csv(os.path.join(Config.output_path, f"submit_{date_time}.csv"), index=False)


In [ ]:
test_021_df.describe()

,0
count,91822.000000
mean,748.341663
std,887.262294
min,61.812490
25%,345.202773
50%,483.229406
75%,789.650426
max,9777.653976
